# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [29]:
# import libraries here

import json
import requests
import pandas as pd

https://docs.github.com/en/free-pro-team@latest/rest
    
data = pd.DataFrame(response.json())
data.head()
username = Laurannvv

SyntaxError: invalid syntax (<ipython-input-29-7166c259fd62>, line 7)

Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [62]:
# your code here


response = requests.get('https://api.github.com/repos/jmolendijk/dataV2-labs/forks')

data = pd.DataFrame(response.json())
data.head(5)

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,forks_count,mirror_url,archived,disabled,open_issues_count,license,forks,open_issues,watchers,default_branch
0,303425358,MDEwOlJlcG9zaXRvcnkzMDM0MjUzNTg=,dataV2-labs,MichelleMartens/dataV2-labs,False,"{'login': 'MichelleMartens', 'id': 72086347, '...",https://github.com/MichelleMartens/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/MichelleMartens/d...,...,0,None,False,False,1,None,0,1,0,master
1,303423312,MDEwOlJlcG9zaXRvcnkzMDM0MjMzMTI=,dataV2-labs,tsharma2020/dataV2-labs,False,"{'login': 'tsharma2020', 'id': 69122280, 'node...",https://github.com/tsharma2020/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/tsharma2020/dataV...,...,0,None,False,False,0,None,0,0,0,master
2,303422412,MDEwOlJlcG9zaXRvcnkzMDM0MjI0MTI=,dataV2-labs,SvetlanaGruzdeva/dataV2-labs,False,"{'login': 'SvetlanaGruzdeva', 'id': 68688714, ...",https://github.com/SvetlanaGruzdeva/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/SvetlanaGruzdeva/...,...,0,None,False,False,0,None,0,0,0,master
3,303406441,MDEwOlJlcG9zaXRvcnkzMDM0MDY0NDE=,dataV2-labs,iswetha522/dataV2-labs,False,"{'login': 'iswetha522', 'id': 8057496, 'node_i...",https://github.com/iswetha522/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/iswetha522/dataV2...,...,1,None,False,False,0,None,1,0,0,master
4,303403314,MDEwOlJlcG9zaXRvcnkzMDM0MDMzMTQ=,dataV2-labs,Sebastieniron/dataV2-labs,False,"{'login': 'Sebastieniron', 'id': 71374088, 'no...",https://github.com/Sebastieniron/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/Sebastieniron/dat...,...,0,None,False,False,0,None,0,0,0,master


#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [55]:
# your code here

from pandas.io.json import json_normalize

results = response.json()
flattened_data = json_normalize(results)

flattened_data.head()



<ipython-input-55-59dafb26d54d>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data = json_normalize(results)


,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,owner.following_url,owner.gists_url,owner.starred_url,owner.subscriptions_url,owner.organizations_url,owner.repos_url,owner.events_url,owner.received_events_url,owner.type,owner.site_admin
0,303425358,MDEwOlJlcG9zaXRvcnkzMDM0MjUzNTg=,dataV2-labs,MichelleMartens/dataV2-labs,False,https://github.com/MichelleMartens/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/MichelleMartens/d...,https://api.github.com/repos/MichelleMartens/d...,...,https://api.github.com/users/MichelleMartens/f...,https://api.github.com/users/MichelleMartens/g...,https://api.github.com/users/MichelleMartens/s...,https://api.github.com/users/MichelleMartens/s...,https://api.github.com/users/MichelleMartens/orgs,https://api.github.com/users/MichelleMartens/r...,https://api.github.com/users/MichelleMartens/e...,https://api.github.com/users/MichelleMartens/r...,User,False
1,303423312,MDEwOlJlcG9zaXRvcnkzMDM0MjMzMTI=,dataV2-labs,tsharma2020/dataV2-labs,False,https://github.com/tsharma2020/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/tsharma2020/dataV...,https://api.github.com/repos/tsharma2020/dataV...,...,https://api.github.com/users/tsharma2020/follo...,https://api.github.com/users/tsharma2020/gists...,https://api.github.com/users/tsharma2020/starr...,https://api.github.com/users/tsharma2020/subsc...,https://api.github.com/users/tsharma2020/orgs,https://api.github.com/users/tsharma2020/repos,https://api.github.com/users/tsharma2020/event...,https://api.github.com/users/tsharma2020/recei...,User,False
2,303422412,MDEwOlJlcG9zaXRvcnkzMDM0MjI0MTI=,dataV2-labs,SvetlanaGruzdeva/dataV2-labs,False,https://github.com/SvetlanaGruzdeva/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/SvetlanaGruzdeva/...,https://api.github.com/repos/SvetlanaGruzdeva/...,...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,https://api.github.com/users/SvetlanaGruzdeva/...,User,False
3,303406441,MDEwOlJlcG9zaXRvcnkzMDM0MDY0NDE=,dataV2-labs,iswetha522/dataV2-labs,False,https://github.com/iswetha522/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/iswetha522/dataV2...,https://api.github.com/repos/iswetha522/dataV2...,...,https://api.github.com/users/iswetha522/follow...,https://api.github.com/users/iswetha522/gists{...,https://api.github.com/users/iswetha522/starre...,https://api.github.com/users/iswetha522/subscr...,https://api.github.com/users/iswetha522/orgs,https://api.github.com/users/iswetha522/repos,https://api.github.com/users/iswetha522/events...,https://api.github.com/users/iswetha522/receiv...,User,False
4,303403314,MDEwOlJlcG9zaXRvcnkzMDM0MDMzMTQ=,dataV2-labs,Sebastieniron/dataV2-labs,False,https://github.com/Sebastieniron/dataV2-labs,copy of the Ironhack data labs for use in JM's...,True,https://api.github.com/repos/Sebastieniron/dat...,https://api.github.com/repos/Sebastieniron/dat...,...,https://api.github.com/users/Sebastieniron/fol...,https://api.github.com/users/Sebastieniron/gis...,https://api.github.com/users/Sebastieniron/sta...,https://api.github.com/users/Sebastieniron/sub...,https://api.github.com/users/Sebastieniron/orgs,https://api.github.com/users/Sebastieniron/repos,https://api.github.com/users/Sebastieniron/eve...,https://api.github.com/users/Sebastieniron/rec...,User,False


In [78]:
response2 = requests.get('https://api.github.com/repos/jmolendijk/dataV2-labs/languages')

results2 = response2.json()

results2


{'Jupyter Notebook': 2471626, 'HTML': 510791, 'Python': 9907}

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [101]:
# your code here
response3 = requests.get('https://api.github.com/repos/Laurannvv/dataV2-labs/commits')

results3 = response3.json()

data3 = pd.DataFrame(response3.json())

fattened_data3 = json_normalize(results3)
data_commit_oct = pd.DataFrame(fattened_data3['commit.author.date'])

data_commit_oct

<ipython-input-101-217e0b8ab543>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  fattened_data3 = json_normalize(results3)


,commit.author.date
0,2020-10-27T11:24:02Z
1,2020-10-27T08:48:50Z
2,2020-10-27T08:47:53Z
3,2020-10-27T08:44:52Z
4,2020-10-26T21:38:44Z
5,2020-10-26T13:50:36Z
6,2020-10-26T13:49:49Z
7,2020-10-26T09:01:09Z
8,2020-10-23T18:12:13Z
9,2020-10-23T08:51:10Z


#### 2. Count how many commit objects are contained in the array.

In [102]:
# your code here

len(data_commit_oct)

30

## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [172]:
# your code here


import re
response_c3 = requests.get('https://api.github.com/repos/ironhack-datalabs/scavenger/contents')

data_c3 = pd.DataFrame(response_c3.json())
data_c3.head(3)

fattened_data_c3 = json_normalize(response_c3.json())
fattened_data_c3


<ipython-input-172-5442d9b93d0c>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  fattened_data_c3 = json_normalize(response_c3.json())


,name,path,sha,size,url,html_url,git_url,download_url,type,_links.self,_links.git,_links.html
0,.gitignore,.gitignore,e43b0f988953ae3a84b00331d0ccf5f7d51cb3cf,10,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,https://raw.githubusercontent.com/ironhack-dat...,file,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
1,15024,15024,2945e51c87ad5da893c954afcf092f06343bbb7d,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
2,15534,15534,5af6f2a7287e4191f39e55693fc1e9c8918d1d3a,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
3,17020,17020,9c49f920aa4d9433fa99a5824128f0e6b90ec5f2,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
4,30351,30351,c488d7f64088c852e22067d48fdc64ee3670f3ba,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
5,40303,40303,30193d9cf62b07bcbb6366513ff03596861f2d29,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
6,44639,44639,22fc3d5c2db80822c351edb2248f3491c8ebda86,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
7,45525,45525,6a4a88cd9084110c8646c3cfd84dfe96b300a4a7,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
8,47222,47222,c7001604cdadc2fe7b82e0f6996690718cac6941,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...
9,47830,47830,f84882ad7560fd2b8c6a0867bc707ce9009ef288,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,https://api.github.com/repos/ironhack-datalabs...,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...


In [177]:
#I did a findall function on every columns to find file name without success.

data_c3['files'] = data_c3['url'].str.findall("\d+.scavengerhunt")
data_c3['files']

0     []
1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
20    []
21    []
22    []
23    []
24    []
25    []
Name: files, dtype: object

#### 2.  Sort the filenames ascendingly.

In [127]:
# your code here

data_c3.sort_values(by=['full_name'])

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,open_issues_count,license,forks,open_issues,watchers,default_branch,temp_clone_token,organization,network_count,subscribers_count
login,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,ironhack-datalabs,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,ironhack-datalabs,5,1
received_events_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
events_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
repos_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
organizations_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
subscriptions_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
starred_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
gists_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
following_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1
followers_url,162375181,MDEwOlJlcG9zaXRvcnkxNjIzNzUxODE=,scavenger,ironhack-datalabs/scavenger,False,https://api.github.com/users/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger,None,False,https://api.github.com/repos/ironhack-datalabs...,...,0,None,5,0,0,master,None,https://api.github.com/users/ironhack-datalabs...,5,1


#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [137]:
# your code hereimport requests



<Response [200]>


#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [ ]:
# your code here

#### 5. Print out the joke.

In [ ]:
# your code here